# Load the user connection data and connect to the SAP HANA database instance

Before running the next cell make sure [../0x00-setup/temp_user.ini](../0x00-setup/temp_user.ini) is copied to [../0x00-setup/user.ini](../0x00-setup/user.ini) and the user + password are set appropriately.

In [ ]:
from hana_ml.algorithms.pal.utility import Settings
myhost, myport, myuser, mypwd = Settings.load_config("../0x00-setup/user.ini")

In [ ]:
from hana_ml import dataframe as hdf
myconn=hdf.ConnectionContext(
    address=myhost, 
    port=myport, 
    user=myuser,
    password=mypwd
)
print(f"Connected to SAP HANA db version {myconn.hana_version()} \nat {myhost}:{myport} as user {myuser}")

In [ ]:
print(myconn.sql("SELECT NOW() FROM DUMMY").collect().CURRENT_TIMESTAMP[0])

# Tables from SAP HANA

In [ ]:
hdf_titanic_train = myconn.table("TRAIN_IMPUTED")

🤓 **Let's discuss**:
- A table we use now and features it has

# [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

In [ ]:
hdf_titanic_train.get_table_structure()

In [ ]:
hdf_titanic_train.head(5).collect()

## New feature 1: Fare per person

Calculate nr of persons per ticket and a fare per person.

In [ ]:
(
    hdf_titanic_train
    .agg([("AVG", "Fare", "AvgFarePerTicket"), ("COUNT", "Ticket", "PersonsPerTicket")], group_by=["Ticket"])
    .select("*", ('ROUND("AvgFarePerTicket"/"PersonsPerTicket", 2)', "FarePerPerson"))
    .describe()
    .collect()
    .iloc[:, :9]
)

Define the reusable procedure, to be applied to both TRAIN and TEST datasets

In [ ]:
def fe_add_fare_per_person(hdf_in: hdf.DataFrame) -> hdf.DataFrame:
    # 1. Calculate nr of persons per ticket and a fare per person
    hdf_ticket_price = hdf_in.agg(
        [("AVG", "Fare", "AvgFarePerTicket"),
         ("COUNT", "Ticket", "PersonsPerTicket")],
        group_by=["Ticket"],
    ).select("*", ('ROUND("AvgFarePerTicket"/"PersonsPerTicket", 2)', "FarePerPerson"))
    
    # 2. Join nr of persons per ticket and a fare per person to the main table with the data
    hdf_out = hdf_in.set_index("Ticket").join(other=hdf_ticket_price.deselect("AvgFarePerTicket").set_index("Ticket"))
    return hdf_out

In [ ]:
hdf_titanic_train_fe1 = fe_add_fare_per_person(hdf_titanic_train)

In [ ]:
hdf_titanic_train_fe1.get_table_structure()

In [ ]:
hdf_titanic_train_fe1 = hdf_titanic_train_fe1.cast({"PersonsPerTicket": "INT"})

### Test table

In [ ]:
hdf_titanic_test = myconn.table("TEST_IMPUTED")

In [ ]:
hdf_titanic_test_fe1 = fe_add_fare_per_person(hdf_titanic_test)

In [ ]:
hdf_titanic_test_fe1.head(5).collect()

In [ ]:
hdf_titanic_test_fe1 = hdf_titanic_test_fe1.cast({"PersonsPerTicket": "INT"})

## New feature 2: First letter of the last name

Define the reusable procedure applied to both TRAIN and TEST datasets

In [ ]:
def fe_add_1letter_lastname (hdf_in: hdf.DataFrame) -> hdf.DataFrame:
    hdf_out = hdf_in.select("*", ('UPPER(LEFT("Name", 1))', "NameFirstLetter"))
    return hdf_out

In [ ]:
hdf_titanic_train_fe2 = fe_add_1letter_lastname(hdf_titanic_train_fe1)

In [ ]:
hdf_titanic_train_fe2.head(5).collect()

In [ ]:
from hana_ml.visualizers.eda import EDAVisualizer

In [ ]:
_, df_letter_distribution = EDAVisualizer().bar_plot(data=hdf_titanic_train_fe2, 
                                                     column='NameFirstLetter', 
                                                     aggregation={'NameFirstLetter':'count'}
                                                    )

### Test table

In [ ]:
hdf_titanic_test_fe2=fe_add_1letter_lastname(hdf_titanic_test_fe1)

In [ ]:
hdf_titanic_test_fe2.head(5).collect()

In [ ]:
_, df_letter_distribution = EDAVisualizer().bar_plot(data=hdf_titanic_test_fe2, 
                                                     column='NameFirstLetter', 
                                                     aggregation={'NameFirstLetter':'count'}
                                                    )

## New feature 3: The title from names

Define the reusable procedure applied to both TRAIN and TEST datasets

In [ ]:
def fe_add_title (hdf_in: hdf.DataFrame) -> hdf.DataFrame:
    hdf_out = hdf_in.select("*", ("SUBSTR_REGEXPR('([A-Za-z\s]+)(?=\.\s)' IN \"Name\")", "Title"))
    return hdf_out

In [ ]:
hdf_titanic_train_fe3 = fe_add_title(hdf_titanic_train_fe2)

In [ ]:
hdf_titanic_train_fe3.head(5).collect()

In [ ]:
from hana_ml.visualizers.eda import EDAVisualizer

In [ ]:
EDAVisualizer().pie_plot(
    data=hdf_titanic_train_fe3,
    column="Title",
    legend=True,
    explode=0,
    startangle=90,
    counterclock=False,
)[1].sort_values(by='COUNT', ascending=False)

### Test table

In [ ]:
hdf_titanic_test_fe3=fe_add_title(hdf_titanic_test_fe2)

In [ ]:
EDAVisualizer().pie_plot(
    data=hdf_titanic_test_fe3,
    column="Title",
    legend=True,
    explode=0,
    startangle=90,
    counterclock=False,
)[1].sort_values(by='COUNT', ascending=False)

### Unified Report for the DataSet

In [ ]:
from hana_ml.visualizers.unified_report import UnifiedReport

In [ ]:
(
    UnifiedReport(hdf_titanic_train_fe3)
    .build(key="PassengerId")
    .display()
)

# Save the new datasets

In [ ]:
hdf_titanic_train_fe3=hdf_titanic_train_fe3.save('TRAIN_FE', force=True)

In [ ]:
hdf_titanic_test_fe3=hdf_titanic_test_fe3.save('TEST_FE', force=True)

# Manual selection of features

In [ ]:
column_id="PassengerId"
column_label="Survived"
features_subset=[
            "NameFirstLetter",
            "FarePerPerson",
            "SibSp",
            "ParCh",
            "Gender",
            "PClass",
            "Embarked",
            "Title",
            "Age",
            "PersonsPerTicket"
]

In [ ]:
# Exclude high-cardinality variables from the train dataset
hdf_titanic_train_v2=myconn.table('TRAIN_FE').select(
            [column_id]+features_subset+[column_label]
        ).cast(column_label, 'NVARCHAR(1)')

In [ ]:
print(hdf_titanic_train_v2.select_statement)

In [ ]:
hdf_titanic_train_v2.head(5).collect()

## RandomDecisionTree

In [ ]:
from hana_ml.algorithms.pal.unified_classification import UnifiedClassification

In [ ]:
uc_rdt_v5 = UnifiedClassification(func="RandomDecisionTree")

In [ ]:
uc_rdt_v5.fit(
    data=hdf_titanic_train_v2,
    key="PassengerId",
    label="Survived",
    training_percent=0.8,
    partition_method="stratified",
    stratified_column="Survived",
    partition_random_state=2
);

## Generate a model report

In [ ]:
from hana_ml.visualizers.unified_report import UnifiedReport
UnifiedReport(uc_rdt_v5).build().display()

## Debrief the model

In [ ]:
from hana_ml.visualizers.model_debriefing import TreeModelDebriefing

In [ ]:
TreeModelDebriefing.tree_debrief_with_dot(uc_rdt_v5.model_[0], iframe_height=700);

## Call predition

In [ ]:
hdf_titanic_test_v2=myconn.table('TEST_FE').select(
            [column_id]+features_subset
        )

In [ ]:
hdf_res = uc_rdt_v5.predict(hdf_titanic_test_v2, key=column_id)

In [ ]:
display(hdf_res.sort("CONFIDENCE", desc=True).collect())

## Compare to the ground truth

In [ ]:
hdf_titanic_complete=myconn.table('COMPLETE', schema='TITANIC')

In [ ]:
hdf_res_ext=hdf_res.set_index('PassengerId').join(hdf_titanic_test_fe3.set_index('PassengerId'))

In [ ]:
hdf_res_incl_groundtruth=(hdf_res_ext.set_index(['Name', 'Ticket']).join(hdf_titanic_complete.set_index(['name', 'ticket']))
                 .select('PassengerId', 'Name', 'Ticket', 'SCORE','survived',('1-ABS(SCORE-"survived")', 'IS_CORRECT'))
                 .cast('SCORE', 'TINYINT')

)

### Accuracy

In [ ]:
hdf_res_incl_groundtruth.select(('SUM("IS_CORRECT")/COUNT("IS_CORRECT")','Accuracy')).collect()

### Confusion matrix

In [ ]:
hdf_res_incl_groundtruth.agg(agg_list=[('count','PassengerId','COUNT')], group_by=['SCORE','survived']).collect()

In [ ]:
df_res_incl_gt_pivot=(hdf_res_incl_groundtruth
 .pivot_table(values='PassengerId', index='SCORE', columns='survived', aggfunc='count')
 .collect()
)

Order columns and rows by 0,1

In [ ]:
(df_res_incl_gt_pivot[[df_res_incl_gt_pivot.columns[0]]+sorted(df_res_incl_gt_pivot.columns[-2:])]
 .sort_values(by=df_res_incl_gt_pivot.columns[0], axis=0)
 .style.hide(axis='index')
)

# Feature selection

Feature selection: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.preprocessing.FeatureSelection.html#featureselection

In PAL: https://help.sap.com/docs/HANA_CLOUD_DATABASE/319d36de4fd64ac3afbf91b1fb3ce8de/29a47ef9bfeb42179aca6723c0ed84b1.html?version=2023_1_QRC&locale=en-US

In [ ]:
from hana_ml.algorithms.pal.preprocessing import FeatureSelection

In [ ]:
hdf_titanic_train_fe3.get_table_structure()

In [ ]:
for fs_method in ['anova', 'chi-squared', 'gini-index', 'fisher-score', 'information-gain', 'MRMR', 'JMI', 'IWFS', 'ADMM']:
    print(f"{fs_method}: ", end="")
    fs = FeatureSelection(fs_method=fs_method, top_k_best=5)
    fs_df = fs.fit_transform(data=hdf_titanic_train_fe3,
                             key='PassengerId',
                             label="Survived",
                             excluded_feature=['Fare','Ticket']
                            )
    print(eval(fs.result_.collect().iloc[0][1])["__SelectedFeatures__"])